In [1]:
import pandas as pd
import numpy as np

In [5]:
# Load dataset
file_path = "Processed_SentimentData.csv"
df = pd.read_csv(file_path)

In [7]:
### 1️⃣ Handling Missing Values
for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna('Unknown', inplace=True)  # Replace missing text with 'Unknown'
    else:
        df[col].fillna(df[col].median(), inplace=True)  # Fill numeric values with median

C:\Users\abcd\AppData\Local\Temp\ipykernel_5888\3405114471.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)  # Fill numeric values with median
C:\Users\abcd\AppData\Local\Temp\ipykernel_5888\3405114471.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [9]:
### 2️⃣ Standardizing Column Names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w]', '', regex=True)

In [11]:
### 3️⃣ Converting Data Types
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_numeric(df[col])  # Convert to numeric if possible
        except ValueError:
            pass  # Keep as text if conversion fails

In [13]:
### 4️⃣ Normalizing Categorical Data
df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)

C:\Users\abcd\AppData\Local\Temp\ipykernel_5888\821254081.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)


In [15]:
### 5️⃣ Date & Time Features
date_columns = [col for col in df.columns if 'date' in col or 'time' in col]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

    # Extract multiple date features
    df[f"{col}_year"] = df[col].dt.year
    df[f"{col}_month"] = df[col].dt.month
    df[f"{col}_day"] = df[col].dt.day
    df[f"{col}_weekday"] = df[col].dt.weekday
    df[f"{col}_week_number"] = df[col].dt.isocalendar().week
    df[f"{col}_quarter"] = df[col].dt.quarter
    df[f"{col}_is_weekend"] = df[col].dt.weekday.isin([5,6]).astype(int)

In [17]:
### 6️⃣ Sentiment Score Processing
sentiment_cols = [col for col in df.columns if 'sentiment' in col]
for col in sentiment_cols:
    df[f"{col}_scaled"] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    df[f"{col}_category"] = pd.cut(df[col], bins=[-1, 0.3, 0.7, 1], labels=['negative', 'neutral', 'positive'])

In [19]:
### 7️⃣ Text Processing Features
text_cols = df.select_dtypes(include=['object']).columns
for col in text_cols:
    df[f"{col}_word_count"] = df[col].apply(lambda x: len(str(x).split()) if isinstance(x, str) else 0)
    df[f"{col}_char_count"] = df[col].apply(lambda x: len(str(x)) if isinstance(x, str) else 0)
    df[f"{col}_avg_word_length"] = df[f"{col}_char_count"] / (df[f"{col}_word_count"] + 1)
    df[f"{col}_special_char_count"] = df[col].apply(lambda x: sum(1 for char in str(x) if not char.isalnum()) if isinstance(x, str) else 0)

In [23]:
### 8️⃣ Numerical Feature Enhancements
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in num_cols:
    df[f"{col}_log"] = np.log1p(df[col])  # Log transformation (to reduce skew)
    df[f"{col}_zscore"] = (df[col] - df[col].mean()) / df[col].std()  # Standardization (z-score)
    df[f"{col}_bin"] = pd.qcut(df[col], q=4, labels=['low', 'medium', 'high', 'very_high'])  # Binning

ValueError: Bin edges must be unique: Index([1.0, 1.0, 1.0, 1.0, 2.0], dtype='float64', name='status_word_count').
You can drop duplicate edges by setting the 'duplicates' kwarg

In [25]:
### 9️⃣ Binary Flags for Missing Values (Before Filling)
for col in df.columns:
    df[f"{col}_missing_flag"] = df[col].isnull().astype(int)

In [27]:
### 🔟 Optimizing Data Size
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [29]:
# Save cleaned dataset
output_path = "PowerBI_Dataset.csv"
df.to_csv(output_path, index=False)
print(f"🚀 Enhanced dataset saved to: {output_path}")

🚀 Enhanced dataset saved to: PowerBI_Dataset.csv
